In [1]:
import os
import pandas as pd
import yaml
import pickle, joblib
import matplotlib.pyplot as plt
from xgboost import XGBRegressor


In [5]:
plot_dct = {
    'seebeck': {'color': '#A3341F', 'label': 'Seebeck'},
    'cond_elec': {'color': '#F1C40F'},
    'cond_therm': {'color': '#3498DB'}
}

In [14]:
def plot_parity(y_true, y_pred, filename):
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(6,12))
    labels = ['Seebeck', 'Conductivity (electronic)', 'Conductivity (ionic)']
    cols = ['seebeck', 'cond_elec', 'cond_therm']
    plt.suptitle(filename)
    for i, ax in enumerate(axes):
        ax.scatter(y_true[cols[i]], y_pred.T[i])
        lim = [min(y_true[cols[i]].min(), y_pred.T[i].min()), max(y_true[cols[i]].max(), y_pred.T[i].max())]
        ax.plot(lim, lim, 'k--', lw=2)
        ax.set_xlim(lim)
        ax.set_ylim(lim)
        ax.set_xlabel(f"True {labels[i]}")
        ax.set_ylabel(f"Predicted {labels[i]}")
    plt.tight_layout()
    plt.savefig(f'{filename}.png')

In [2]:
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

In [3]:
module_dir = '/Users/yujinkang/AMW25'
data_dir = '/Users/yujinkang/AMW25/data'
scaler = joblib.load(f'{data_dir}/scaler.pkl')

In [4]:
X_train_mat = pd.read_csv(f'{data_dir}/X_train_mat.csv')
X_train_tot = pd.read_csv(f'{data_dir}/X_train.csv')
y_train = pd.read_csv(f'{data_dir}/unscaled/y_train.csv').drop(columns=['index'])
y_test = pd.read_csv(f'{data_dir}/unscaled/y_test.csv').drop(columns=['index'])
X_val_mat = pd.read_csv(f'{data_dir}/X_eval_mat.csv')
X_val_tot = pd.read_csv(f'{data_dir}/X_eval.csv')
y_val = pd.read_csv(f'{data_dir}/unscaled/y_eval.csv').drop(columns=['index'])
X_test_tot = pd.read_csv(f'{data_dir}/X_test.csv')
X_test_mat = pd.read_csv(f'{data_dir}/X_test_mat.csv')
config_dir = '/Users/yujinkang/AMW25/config.yaml'
with open(config_dir, 'r') as file:
    config = yaml.safe_load(file)

targets = config['data']['target']
syms = config['data']['sym']

y_train_scaled = scaler.fit_transform(y_train).T
y_test_scaled = scaler.transform(y_test).T
y_val_scaled = scaler.transform(y_val).T

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yujinkang/AMW25/config.yaml'

In [124]:
dat = {
    'X_train_tot': X_train_tot.copy(),
    'X_eval_tot': X_val_tot.copy(),
    'X_test_tot': X_test_tot.copy(),
    'y_train': y_train.copy(),
    'y_eval': y_val.copy(),
    'y_test': y_test.copy(),
    'X_train_mat': X_train_mat.copy(),
    'X_eval_mat': X_val_mat.copy(),
    'X_test_mat': X_test_mat.copy(),
    'y_train_scaled': y_train_scaled.copy(),
    'y_eval_scaled': y_val_scaled.copy(),
    'y_test_scaled': y_test_scaled.copy(),
}

In [125]:
with open('/Users/dnjf/AMW25/tutorial/data_to_load.pkl', 'wb') as f:
    pickle.dump(dat, f)

In [5]:
with open('/Users/yujinkang/AMW25/tutorial/DataToLoad.pkl','rb') as f:
        data = pickle.load(f)

In [10]:
import numpy as np
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [11]:
import math

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

def scorer(y_test, y_pred):
    mae_s = math.log(mean_absolute_error(y_test['seebeck'], y_pred.T[0]))
    mae_o = math.log10(mean_absolute_error(y_test['cond_elec'], y_pred.T[1]))
    mae_k = mean_absolute_error(y_test['cond_therm'], y_pred.T[2])

    mae = (mae_s + mae_o + mae_k) / 3
    return mae

def mape_scorer(y_test, y_pred):
    mape_s = mape(y_test['seebeck'], y_pred.T[0])
    mape_o = mape(y_test['cond_elec'], y_pred.T[1])
    mape_k = mape(y_test['cond_therm'], y_pred.T[2])

    mape = (mape_s + mape_o + mape_k) / 3
    return mape

def objective(trial):
    X_train = data['X_train_mat']
    X_test = data['X_test_mat']
    y_train = data['y_train']
    y_test = data['y_test']

    n_estimators = trial.suggest_int("n_estimators", 400, 3000, step=100) 
    max_depth = trial.suggest_int("max_depth", 1, 15)
    learning_rate = trial.suggest_float("learning_rate", 0.02, 0.3, log=True)  
    reg_lambda = trial.suggest_float("reg_lambda", 0.3, 0.6) 
    reg_alpha = trial.suggest_float("reg_alpha", 0.3, 0.6)  
    gamma = trial.suggest_float("gamma", 0, 10)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 15)

    model = XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        gamma=gamma,
        min_child_weight=min_child_weight,
        early_stopping_rounds=200,
    )
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    y_pred = model.predict(X_test)

    accuracy = scorer(y_test,y_pred)
    trial.set_user_attr("accuracy", accuracy)

    return accuracy

class StopWhenAccuracyOverCrit:  
    def __init__(self, crit): 
        self.crit = crit 

    def __call__(self, study, trial) -> None:        
        if trial.user_attrs["accuracy"] > self.crit:  
            study.stop() 
            
class StopWhenAccuracyUnderCrit:  
    def __init__(self, crit): 
        self.crit = crit 

    def __call__(self, study, trial) -> None:        
        if trial.user_attrs["accuracy"] < self.crit:  
            study.stop() 
            
# Define function for model selection 

def xgbr_trainer(study, model_name, X_train, y_train, X_test, y_test, X_eval, y_eval):
    model = XGBRegressor(**study.best_params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    y_pred_train = model.predict(X_train) 
    y_pred_test = model.predict(X_test) 
    y_pred_eval = model.predict(X_eval)

    data = {'y_train': y_train, 'y_pred_train': y_pred_train.T,
            'y_test': y_test, 'y_pred_test': y_pred_test.T,
            'y_eval': y_eval, 'y_pred_eval': y_pred_eval.T}

    r2_s = r2_score(y_train['seebeck'], y_pred_train.T[0])
    r2_o = r2_score(y_train['cond_elec'], y_pred_train.T[1])
    r2_k = r2_score(y_train['cond_therm'], y_pred_train.T[2])

    mae_s = mean_absolute_error(y_train['seebeck'], y_pred_train.T[0])
    mae_o = mean_absolute_error(y_train['cond_elec'], y_pred_train.T[1])
    mae_k = mean_absolute_error(y_train['cond_therm'], y_pred_train.T[2])


    print(f"r2_score of seebeck at train set: {round(r2_s, 3)}")
    print(f"r2_score of cond_elec at train set: {round(r2_o, 3)}")
    print(f"r2_score of cond_therm at train set: {round(r2_k, 3)}")

    print(f"mae_score of seebeck at train set: {round(mae_s, 3)}")
    print(f"mae_score of cond_elec at train set: {round(mae_o, 3)}")
    print(f"mae_score of cond_therm at train set: {round(mae_k, 3)}")


    r2_s_test = r2_score(y_test['seebeck'], y_pred_test.T[0])
    r2_o_test = r2_score(y_test['cond_elec'], y_pred_test.T[1])
    r2_k_test = r2_score(y_test['cond_therm'], y_pred_test.T[2])

    mae_s_test = mean_absolute_error(y_test['seebeck'], y_pred_test.T[0])
    mae_o_test = mean_absolute_error(y_test['cond_elec'], y_pred_test.T[1])
    mae_k_test = mean_absolute_error(y_test['cond_therm'], y_pred_test.T[2])


    print(f"r2_score of seebeck at test set: {round(r2_s_test, 3)}")
    print(f"r2_score of cond_elec at test set: {round(r2_o_test, 3)}")
    print(f"r2_score of cond_therm at test set: {round(r2_k_test, 3)}")

    print(f"mae_score of seebeck at test set: {round(mae_s_test, 3)}")
    print(f"mae_score of cond_elec at test set: {round(mae_o_test, 3)}")
    print(f"mae_score of cond_therm at test set: {round(mae_k_test, 3)}")


    r2_s_eval = r2_score(y_eval['seebeck'], y_pred_eval.T[0])
    r2_o_eval = r2_score(y_eval['cond_elec'], y_pred_eval.T[1])
    r2_k_eval = r2_score(y_eval['cond_therm'], y_pred_eval.T[2])

    mae_s_eval = mean_absolute_error(y_eval['seebeck'], y_pred_eval.T[0])
    mae_o_eval = mean_absolute_error(y_eval['cond_elec'], y_pred_eval.T[1])
    mae_k_eval = mean_absolute_error(y_eval['cond_therm'], y_pred_eval.T[2])


    print(f"r2_score of seebeck at eval set: {round(r2_s_eval, 3)}")
    print(f"r2_score of cond_elec at eval set: {round(r2_o_eval, 3)}")
    print(f"r2_score of cond_therm at eval set: {round(r2_k_eval, 3)}")

    print(f"mae_score of seebeck at eval set: {round(mae_s_eval, 3)}")
    print(f"mae_score of cond_elec at eval set: {round(mae_o_eval, 3)}")
    print(f"mae_score of cond_therm at eval set: {round(mae_k_eval, 3)}")



    with open(f'{model_name}_data.pkl', 'wb') as file:
        pickle.dump(data, file)


    plot_parity(y_train, y_pred_train, f'{model_name}_train')
    plot_parity(y_test, y_pred_test, f'{model_name}_test')
    plot_parity(y_eval, y_pred_eval, f'{model_name}_eval')

    joblib.dump(model, f'{model_name}_joblib.pkl')

    return model

In [16]:
import optuna
stop_search = StopWhenAccuracyUnderCrit(50)
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=3))
study.optimize(objective, callbacks=[stop_search])

[I 2025-08-22 05:50:07,460] A new study created in memory with name: no-name-d95b1f3b-65fd-48be-b025-b3cdaec7daea
[I 2025-08-22 05:50:08,872] Trial 0 finished with value: 5054.161248681524 and parameters: {'n_estimators': 1800, 'max_depth': 11, 'learning_rate': 0.0439704101134468, 'reg_lambda': 0.45324828155929886, 'reg_alpha': 0.5678840863042964, 'gamma': 8.96293088933438, 'min_child_weight': 2}. Best is trial 0 with value: 5054.161248681524.
[I 2025-08-22 05:50:09,438] Trial 1 finished with value: 165.23141137812488 and parameters: {'n_estimators': 900, 'max_depth': 1, 'learning_rate': 0.06598764049744814, 'reg_lambda': 0.3089628632635701, 'reg_alpha': 0.43704996731841333, 'gamma': 6.491440476147607, 'min_child_weight': 5}. Best is trial 1 with value: 165.23141137812488.
[I 2025-08-22 05:50:10,395] Trial 2 finished with value: 5374.549409047644 and parameters: {'n_estimators': 2200, 'max_depth': 9, 'learning_rate': 0.02134198832067715, 'reg_lambda': 0.46765622639726456, 'reg_alpha': 

In [17]:
study.best_params
print("Best parameters:", study.best_params)

model = xgbr_trainer(study=study, model_name='mat_50', X_train=data['X_train_mat'], y_train=data['y_train'], X_test=data['X_test_mat'], y_test=data['y_test'], X_eval=data['X_eval_mat'], y_eval=data['y_eval'])

Best parameters: {'n_estimators': 2100, 'max_depth': 1, 'learning_rate': 0.05781306153314478, 'reg_lambda': 0.4888595362392452, 'reg_alpha': 0.3365294943099512, 'gamma': 6.498332782993996, 'min_child_weight': 1}
[0]	validation_0-rmse:112913.90087
[1]	validation_0-rmse:111474.22114


[2]	validation_0-rmse:110227.40458
[3]	validation_0-rmse:109153.48445
[4]	validation_0-rmse:108233.94955
[5]	validation_0-rmse:107451.72527
[6]	validation_0-rmse:106791.17840
[7]	validation_0-rmse:105900.18394
[8]	validation_0-rmse:105438.76658
[9]	validation_0-rmse:104779.05200
[10]	validation_0-rmse:104474.63589
[11]	validation_0-rmse:103999.20803
[12]	validation_0-rmse:103816.79801
[13]	validation_0-rmse:103486.56730
[14]	validation_0-rmse:103397.54724
[15]	validation_0-rmse:103180.44310
[16]	validation_0-rmse:103161.72019
[17]	validation_0-rmse:103031.72489
[18]	validation_0-rmse:103064.87491
[19]	validation_0-rmse:103001.09922
[20]	validation_0-rmse:103071.58944
[21]	validation_0-rmse:103057.41196
[22]	validation_0-rmse:103153.88236
[23]	validation_0-rmse:103176.18020
[24]	validation_0-rmse:103289.84523
[25]	validation_0-rmse:103407.81270
[26]	validation_0-rmse:103464.85111
[27]	validation_0-rmse:103590.44364
[28]	validation_0-rmse:103138.44909
[29]	validation_0-rmse:103220.44182


ValueError: feature_names mismatch: ['n_rot', 'n_inv', 'n_rotoinv', 'n_mirror', 'n_rot_screw', '1/2tran_screw', '1/3tran_screw', '3/4tran_screw', '1/4tran_screw', '1/6tran_screw', 'n_glide', '1/2tran_glide', '3/4tran_glide', '1/4tran_glide', 'n_tr'] ['MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData mean Number', 'MagpieData mode Number', 'MagpieData mean MendeleevNumber', 'MagpieData mode MendeleevNumber', 'MagpieData mean AtomicWeight', 'MagpieData mode AtomicWeight', 'MagpieData mean MeltingT', 'MagpieData mode MeltingT', 'MagpieData mean Column', 'MagpieData minimum CovalentRadius', 'MagpieData maximum CovalentRadius', 'MagpieData mean CovalentRadius', 'MagpieData minimum Electronegativity', 'MagpieData maximum Electronegativity', 'MagpieData range Electronegativity', 'MagpieData mean Electronegativity', 'MagpieData mean NsValence', 'MagpieData avg_dev NsValence', 'MagpieData mode NsValence', 'MagpieData mean NpValence', 'MagpieData avg_dev NpValence', 'MagpieDa

In [19]:
with open('mat_study.pkl', 'wb') as file:
    pickle.dump(study, file)
    

In [21]:
with open('mat_study.pkl', 'rb') as file:
    st = pickle.load(file)

In [26]:
model = XGBRegressor(**st.best_params)
model.fit(data['X_train_mat'], data['y_train'], eval_set=[(data['X_test_mat'], data['y_test'])])

[0]	validation_0-rmse:112913.90087
[1]	validation_0-rmse:111474.22114
[2]	validation_0-rmse:110227.40458
[3]	validation_0-rmse:109153.48445
[4]	validation_0-rmse:108233.94955
[5]	validation_0-rmse:107451.72527
[6]	validation_0-rmse:106791.17840
[7]	validation_0-rmse:105900.18394
[8]	validation_0-rmse:105438.76658
[9]	validation_0-rmse:104779.05200
[10]	validation_0-rmse:104474.63589
[11]	validation_0-rmse:103999.20803
[12]	validation_0-rmse:103816.79801
[13]	validation_0-rmse:103486.56730
[14]	validation_0-rmse:103397.54724
[15]	validation_0-rmse:103180.44310
[16]	validation_0-rmse:103161.72019
[17]	validation_0-rmse:103031.72489
[18]	validation_0-rmse:103064.87491
[19]	validation_0-rmse:103001.09922
[20]	validation_0-rmse:103071.58944
[21]	validation_0-rmse:103057.41196
[22]	validation_0-rmse:103153.88236
[23]	validation_0-rmse:103176.18020
[24]	validation_0-rmse:103289.84523
[25]	validation_0-rmse:103407.81270
[26]	validation_0-rmse:103464.85111
[27]	validation_0-rmse:103590.44364
[2

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [27]:
joblib.dump(model, 'mat_model.pkl')

['mat_model.pkl']

ValueError: feature_names mismatch: ['n_rot', 'n_inv', 'n_rotoinv', 'n_mirror', 'n_rot_screw', '1/2tran_screw', '1/3tran_screw', '3/4tran_screw', '1/4tran_screw', '1/6tran_screw', 'n_glide', '1/2tran_glide', '3/4tran_glide', '1/4tran_glide', 'n_tr'] ['MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData mean Number', 'MagpieData mode Number', 'MagpieData mean MendeleevNumber', 'MagpieData mode MendeleevNumber', 'MagpieData mean AtomicWeight', 'MagpieData mode AtomicWeight', 'MagpieData mean MeltingT', 'MagpieData mode MeltingT', 'MagpieData mean Column', 'MagpieData minimum CovalentRadius', 'MagpieData maximum CovalentRadius', 'MagpieData mean CovalentRadius', 'MagpieData minimum Electronegativity', 'MagpieData maximum Electronegativity', 'MagpieData range Electronegativity', 'MagpieData mean Electronegativity', 'MagpieData mean NsValence', 'MagpieData avg_dev NsValence', 'MagpieData mode NsValence', 'MagpieData mean NpValence', 'MagpieData avg_dev NpValence', 'MagpieDa

In [40]:
sym= ['n_rot','n_inv','n_rotoinv', 'n_mirror', 'n_rot_screw','1/2tran_screw','1/3tran_screw','3/4tran_screw','1/4tran_screw','1/6tran_screw','n_glide','1/2tran_glide','3/4tran_glide','1/4tran_glide','n_tr']

X_eval = data['X_eval_tot'][sym]
data['y_eval']

,seebeck,cond_elec,cond_therm
0,-125.0,18010.0,0.62
1,59.0,17350.0,0.50
2,204.0,623.0,0.62
3,556.0,2.0,0.42
4,289.0,3013.0,0.68
5,114.0,51194.0,2.18
6,-13.0,551.0,2.08
7,149.0,90185.0,1.42
8,195.0,14865.0,1.25
9,50.0,19826.0,0.53


In [41]:
y_eval_predict = model.predict(X_eval)

In [42]:
y_eval = data['y_eval']

In [43]:
y_eval_predict

array([[ 3.01384277e+01,  3.20696973e+04,  1.08817232e+00],
       [ 3.05822086e+01,  1.81633047e+04,  2.22126985e+00],
       [ 2.16051208e+02,  1.71821074e+04,  1.08817232e+00],
       [ 3.01384277e+01,  3.20696973e+04,  1.08817232e+00],
       [ 3.01384277e+01,  3.20696973e+04,  1.08817232e+00],
       [ 1.89922226e+02,  1.60801016e+05,  2.34898186e+00],
       [-6.60681305e+01,  1.20618867e+05,  2.50120354e+00],
       [ 9.41429749e+01,  1.17304664e+05,  2.34898186e+00],
       [ 4.39415398e+01,  2.58946344e+05,  3.34947801e+00],
       [ 3.05822086e+01,  1.81633047e+04,  2.22126985e+00],
       [-2.19432617e+02,  7.47369873e+03,  6.24785328e+00],
       [ 4.39415398e+01,  2.58946344e+05,  3.34947801e+00],
       [ 6.67202234e+00,  1.16221242e+05,  1.08817232e+00],
       [ 4.24004059e+01,  3.51781500e+05,  8.34182358e+00],
       [ 3.01384277e+01,  3.20696973e+04,  1.08817232e+00],
       [ 4.39415398e+01,  2.58946344e+05,  3.34947801e+00],
       [ 4.24004059e+01,  3.51781500e+05

In [44]:
joblib.dump(model, 'mat_model.pkl')

['mat_model.pkl']

In [45]:
data2 = {
    'X_eval': X_eval,
    'y_eval': y_eval,
    'y_eval_predict': y_eval_predict.T
}

In [49]:
with open('mat_eval.pkl', 'wb') as file:
    pickle.dump(data2, file)


In [48]:
y_eval_predict

array([[ 3.01384277e+01,  3.20696973e+04,  1.08817232e+00],
       [ 3.05822086e+01,  1.81633047e+04,  2.22126985e+00],
       [ 2.16051208e+02,  1.71821074e+04,  1.08817232e+00],
       [ 3.01384277e+01,  3.20696973e+04,  1.08817232e+00],
       [ 3.01384277e+01,  3.20696973e+04,  1.08817232e+00],
       [ 1.89922226e+02,  1.60801016e+05,  2.34898186e+00],
       [-6.60681305e+01,  1.20618867e+05,  2.50120354e+00],
       [ 9.41429749e+01,  1.17304664e+05,  2.34898186e+00],
       [ 4.39415398e+01,  2.58946344e+05,  3.34947801e+00],
       [ 3.05822086e+01,  1.81633047e+04,  2.22126985e+00],
       [-2.19432617e+02,  7.47369873e+03,  6.24785328e+00],
       [ 4.39415398e+01,  2.58946344e+05,  3.34947801e+00],
       [ 6.67202234e+00,  1.16221242e+05,  1.08817232e+00],
       [ 4.24004059e+01,  3.51781500e+05,  8.34182358e+00],
       [ 3.01384277e+01,  3.20696973e+04,  1.08817232e+00],
       [ 4.39415398e+01,  2.58946344e+05,  3.34947801e+00],
       [ 4.24004059e+01,  3.51781500e+05